In [79]:
import pandas as pd 
import json
import requests
from bs4 import BeautifulSoup
import numpy as np

In [80]:
#Import the table of the citis/towns from html
res = requests.get('https://en.wikipedia.org/wiki/List_of_municipalities_in_Massachusetts')
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
raw = pd.read_html(str(table))
df_raw = raw[0]
df_raw.head()

,Municipality,Type[1],County[2],Form of government[1],Population (2010)[2],Year established[3]
0,Abington,Town,Plymouth,Open town meeting,15985,1712
1,Acton,Town,Middlesex,Open town meeting,21924,1735
2,Acushnet,Town,Bristol,Open town meeting,10303,1860
3,Adams,Town,Berkshire,Representative town meeting,8485,1778
4,Agawam,City[4],Hampden,Mayor-council,28438,1855


In [81]:
#Clean and uniform the dataframe to leave only the data we need
#Include only cities
df_cities = df_raw[(df_raw['Type[1]'].str.contains('City'))]
df_cities.head()

,Municipality,Type[1],County[2],Form of government[1],Population (2010)[2],Year established[3]
4,Agawam,City[4],Hampden,Mayor-council,28438,1855
6,Amesbury,City,Essex,Mayor-council,16283,1668
16,Attleboro,City,Bristol,Mayor-council,43593,1694
20,Barnstable,City[4],Barnstable,Council-manager,45193,1639
30,Beverly,City,Essex,Mayor-council,39502,1668


In [82]:
df_cities.shape

(58, 6)

In [83]:
df_cities.drop(['Type[1]', 'County[2]', 'Form of government[1]', 'Population (2010)[2]', 'Year established[3]'], axis =1, inplace = True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [84]:
df_cities.rename(columns = {'Municipality': 'City'}, inplace = True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [85]:
df_cities.head()

,City
4,Agawam
6,Amesbury
16,Attleboro
20,Barnstable
30,Beverly


### Add in Geographical Coordinates

In [91]:
from geopy.geocoders import Nominatim 

In [92]:
# create empty lists
list_lat = []   
list_long = []

# iterate over rows in dataframe
geolocator = Nominatim(user_agent = 'ma_explorer', timeout=10)
for index, row in df_cities.iterrows():
    Cityname = row["City"]
    State = 'MA'      
    address = str(Cityname) +', '+ str(State)    
    location = geolocator.geocode(address)
    if location:
        lat = location.latitude
        long = location.longitude
        list_lat.append(lat)
        list_long.append(long)
    else:
        list_lat.append('0')
        list_long.append('0') 
    
# create new columns from lists    
df_cities['Latitude'] = list_lat   
df_cities['Longitude'] = list_long    

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [93]:
df_cities.head()

,City,Latitude,Longitude
4,Agawam,42.069539,-72.614812
6,Amesbury,42.857954,-70.930092
16,Attleboro,41.944544,-71.285608
20,Barnstable,41.700113,-70.299467
30,Beverly,42.558428,-70.880049


### Define Foursquare Credentials and Version

In [94]:
CLIENT_ID = 'L21RGYF5KV2EHC2NNUZVKZP0G2LZ5Z0B5I1UWDWLHFS00NUS' 
CLIENT_SECRET = 'TVC1C0OCORZNYQ4NSI2GXFCUAFKF5QHL51F2RWWQODET3CNM' 

# Foursquare API version
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L21RGYF5KV2EHC2NNUZVKZP0G2LZ5Z0B5I1UWDWLHFS00NUS
CLIENT_SECRET:TVC1C0OCORZNYQ4NSI2GXFCUAFKF5QHL51F2RWWQODET3CNM


### Explore Neighborhoods in MA

In [95]:
# limit of number of venues returned by Foursquare API
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Run the above function on each cities and create a new dataframe called MA_venues.

In [96]:
ma_venues = getNearbyVenues(names = df_cities['City'], 
                                 latitudes = df_cities['Latitude'], 
                                 longitudes = df_cities['Longitude']
                                  )

Agawam
Amesbury
Attleboro
Barnstable
Beverly
Boston
Braintree
Bridgewater
Brockton
Cambridge
Chelsea
Chicopee
East Longmeadow
Easthampton
Everett
Fall River
Fitchburg
Framingham
Franklin
Gardner
Gloucester
Greenfield
Haverhill
Holyoke
Lawrence
Leominster
Lowell
Lynn
Malden
Marlborough
Medford
Melrose
Methuen
New Bedford
Newburyport
Newton
North Adams
North Attleborough
Northampton
Palmer
Peabody
Pittsfield
Quincy
Randolph
Revere
Salem
Somerville
Southbridge
Springfield
Taunton
Waltham
Watertown
West Springfield
Westfield
Weymouth
Winthrop
Woburn
Worcester


### Check the Resulting Dataframe

In [97]:
#Check the size of the dataframe
print(ma_venues.shape)
ma_venues.head()

(1281, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agawam,42.069539,-72.614812,Giovanni's,42.069010,-72.615021,Bakery
1,Agawam,42.069539,-72.614812,Phelp's School Field,42.070885,-72.613872,Baseball Field
2,Agawam,42.069539,-72.614812,Dealmaster Pro,42.069589,-72.615128,Business Service
3,Agawam,42.069539,-72.614812,Utopia,42.069134,-72.614888,Food & Drink Shop
4,Agawam,42.069539,-72.614812,Shell,42.072247,-72.615099,Gas Station


In [98]:
#Check how many venues were returned for each of the cities
ma_venues.groupby('City').count().head()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Agawam,6,6,6,6,6,6
Amesbury,27,27,27,27,27,27
Attleboro,16,16,16,16,16,16
Barnstable,8,8,8,8,8,8
Beverly,15,15,15,15,15,15


In [99]:
#Check how many unique categories from all the returned venues
print('There are {} uniques categories.'.format(len(ma_venues['Venue Category'].unique())))

There are 227 uniques categories.


### Analyze Each City

In [100]:
# One hot encoding
ma_onehot = pd.get_dummies(ma_venues[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
ma_onehot['City'] = ma_venues['City'] 

# Move neighborhood column to the first column
fixed_columns = [ma_onehot.columns[-1]] + list(ma_onehot.columns[:-1])
ma_onehot = ma_onehot[fixed_columns]

ma_onehot.head()

,City,ATM,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Caribbean Restaurant,Casino,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food & Drink Shop,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motorsports Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Office,Optical Shop,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parking,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool Hall,Portuguese Restaurant,Pub,Record Shop,Rental Car Location,Rental Service,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Soccer Field,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Food Gathering,Strip Club,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agawam,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Agawam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [101]:
#I would like to have a look at all the categories
from IPython.display import display
pd.options.display.max_columns = None
display(ma_onehot)

,City,ATM,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Caribbean Restaurant,Casino,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food & Drink Shop,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motorsports Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Office,Optical Shop,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parking,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool Hall,Portuguese Restaurant,Pub,Record Shop,Rental Car Location,Rental Service,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Soccer Field,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Food Gathering,Strip Club,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agawam,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Agawam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [102]:
ma_onehot.set_index('City', inplace = True)

In [103]:
#Clean the dataframe to leave only the columns contains 'restuartant'
ma_onehot_rest = ma_onehot.filter(regex = 'Restaurant')
ma_onehot_rest.head(10)

,African Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cambodian Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Persian Restaurant,Portuguese Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Spanish Restaurant,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agawam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Agawam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Agawam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Agawam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Agawam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Agawam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Amesbury,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Amesbury,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Amesbury,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [104]:
#Group rows by cities and by taking the mean of the frequency of occurrence of each category
ma_grouped = ma_onehot_rest.groupby('City').mean().reset_index()
ma_grouped.head(10)

,City,African Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cambodian Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,New American Restaurant,Persian Restaurant,Portuguese Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Spanish Restaurant,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Agawam,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.000000,0.0
1,Amesbury,0.0,0.148148,0.000000,0.00,0.0,0.0,0.0,0.0,0.074074,0.037037,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.000000,0.037037,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.037037,0.0
2,Attleboro,0.0,0.000000,0.062500,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.000000,0.062500,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000,0.0,0.0625,0.0,0.0,0.000000,0.000000,0.000000,0.0
3,Barnstable,0.0,0.125000,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.125,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.000000,0.0
4,Beverly,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.000000,0.0
5,Boston,0.0,0.020000,0.000000,0.01,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.02,0.01,0.0,0.0,0.02,0.0,0.000000,0.050000,0.0,0.0,0.0,0.0,0.020000,0.000000,0.0,0.000000,0.0,0.010000,0.0,0.0,0.02,0.090,0.0,0.0000,0.0,0.0,0.000000,0.010000,0.010000,0.0
6,Braintree,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.000000,0.0
7,Bridgewater,0.0,0.100000,0.000000,0.00,0.0,0.0,0.0,0.0,0.050000,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.000000,0.050000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000,0.0,0.0000,0.0,0.0,0.000000,0.050000,0.000000,0.0
8,Brockton,0.0,0.000000,0.166667,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.000000,0.0
9,Cambridge,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.054054,0.027027,0.0,0.0,0.0,0.0,0.027027,0.027027,0.0,0.027027,0.0,0.027027,0.0,0.0,0.00,0.000,0.0,0.0000,0.0,0.0,0.027027,0.027027,0.000000,0.0


In [105]:
#Print each cities along with the top 10 most common venues
num_top_venues = 10

for hood in ma_grouped['City']:
    print("----"+hood+"----")
    temp = ma_grouped[ma_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agawam----
                       venue  freq
0         African Restaurant   0.0
1      Portuguese Restaurant   0.0
2  Latin American Restaurant   0.0
3   Mediterranean Restaurant   0.0
4         Mexican Restaurant   0.0
5            Meze Restaurant   0.0
6  Middle Eastern Restaurant   0.0
7        Moroccan Restaurant   0.0
8    New American Restaurant   0.0
9         Persian Restaurant   0.0


----Amesbury----
                           venue  freq
0            American Restaurant  0.15
1             Chinese Restaurant  0.07
2  Vegetarian / Vegan Restaurant  0.04
3             Italian Restaurant  0.04
4        Comfort Food Restaurant  0.04
5             African Restaurant  0.00
6          Portuguese Restaurant  0.00
7             Mexican Restaurant  0.00
8                Meze Restaurant  0.00
9      Middle Eastern Restaurant  0.00


----Attleboro----
                       venue  freq
0           Asian Restaurant  0.06
1         Italian Restaurant  0.06
2         Spanish Restauran

                     venue  freq
0      American Restaurant  0.06
1       Chinese Restaurant  0.03
2       Mexican Restaurant  0.03
3  New American Restaurant  0.03
4     Fast Food Restaurant  0.03
5       Seafood Restaurant  0.03
6               Restaurant  0.03
7          Thai Restaurant  0.03
8       Spanish Restaurant  0.00
9         Sushi Restaurant  0.00


----Greenfield----
                       venue  freq
0        American Restaurant  0.06
1            Thai Restaurant  0.03
2         Mexican Restaurant  0.03
3           Greek Restaurant  0.03
4         Persian Restaurant  0.00
5  Latin American Restaurant  0.00
6   Mediterranean Restaurant  0.00
7            Meze Restaurant  0.00
8  Middle Eastern Restaurant  0.00
9        Moroccan Restaurant  0.00


----Haverhill----
                       venue  freq
0         Seafood Restaurant  0.06
1        American Restaurant  0.06
2         African Restaurant  0.00
3         Persian Restaurant  0.00
4  Latin American Restaurant  0.00
5

                       venue  freq
0         Italian Restaurant  0.12
1        American Restaurant  0.06
2         Mexican Restaurant  0.06
3          Kosher Restaurant  0.00
4  Latin American Restaurant  0.00
5   Mediterranean Restaurant  0.00
6            Meze Restaurant  0.00
7  Middle Eastern Restaurant  0.00
8        Moroccan Restaurant  0.00
9    New American Restaurant  0.00


----Salem----
                      venue  freq
0        Mexican Restaurant  0.04
1   New American Restaurant  0.03
2         Korean Restaurant  0.01
3        Seafood Restaurant  0.01
4       American Restaurant  0.01
5        Italian Restaurant  0.01
6         Indian Restaurant  0.01
7  Mediterranean Restaurant  0.01
8                Restaurant  0.01
9          Sushi Restaurant  0.01


----Somerville----
                       venue  freq
0          Indian Restaurant  0.04
1            Meze Restaurant  0.04
2    New American Restaurant  0.04
3         African Restaurant  0.00
4      Portuguese Restaurant 

In [106]:
#Put the above into a new dataframe

#Function to sort the value in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#Create the new dataframe and display the top 10 venues for each neighbourhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = ma_grouped['City']

for ind in np.arange(ma_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ma_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agawam,Vietnamese Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
1,Amesbury,American Restaurant,Chinese Restaurant,Comfort Food Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cambodian Restaurant,Caribbean Restaurant,Belgian Restaurant
2,Attleboro,Asian Restaurant,Spanish Restaurant,Italian Restaurant,Vietnamese Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant
3,Barnstable,American Restaurant,Seafood Restaurant,Vietnamese Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant
4,Beverly,Vietnamese Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant


### Cluster MA Cities

In [107]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium 

In [108]:
# Run k-means to cluster the citis into 5 clusters

# Set number of clusters
kclusters = 5

ma_grouped_clustering = ma_grouped.drop('City', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ma_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 0, 1, 1, 1, 0, 1, 1], dtype=int32)

In [109]:
#Create a new dataframe that includes the cluster as well as the top 10 types of restaurants for each cities

# Add clustering labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ma_merged = df_cities

# Merge ma_grouped with ma_data to add latitude/longitude for each city
ma_merged = ma_merged.join(city_venues_sorted.set_index('City'), on='City')

ma_merged

,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Agawam,42.069539,-72.614812,1.0,Vietnamese Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
6,Amesbury,42.857954,-70.930092,0.0,American Restaurant,Chinese Restaurant,Comfort Food Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cambodian Restaurant,Caribbean Restaurant,Belgian Restaurant
16,Attleboro,41.944544,-71.285608,1.0,Asian Restaurant,Spanish Restaurant,Italian Restaurant,Vietnamese Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant
20,Barnstable,41.700113,-70.299467,0.0,American Restaurant,Seafood Restaurant,Vietnamese Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant
30,Beverly,42.558428,-70.880049,1.0,Vietnamese Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
35,Boston,42.360253,-71.058291,1.0,Seafood Restaurant,Italian Restaurant,American Restaurant,Restaurant,Falafel Restaurant,Greek Restaurant,Mediterranean Restaurant,New American Restaurant,Thai Restaurant,Belgian Restaurant
40,Braintree,42.222322,-70.999492,1.0,Vietnamese Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
42,Bridgewater,41.990379,-70.975043,0.0,American Restaurant,Chinese Restaurant,Thai Restaurant,Italian Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant
44,Brockton,42.083433,-71.018379,1.0,Asian Restaurant,Vietnamese Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant
49,Cambridge,42.375100,-71.105616,1.0,Indian Restaurant,New American Restaurant,Middle Eastern Restaurant,Thai Restaurant,Tapas Restaurant,Italian Restaurant,Mediterranean Restaurant,Mexican Restaurant,French Restaurant,Fast Food Restaurant


In [110]:
ma_merged.shape

(58, 14)

In [111]:
ma_merged = ma_merged[pd.notnull(ma_merged['Cluster Labels'])]

In [112]:
ma_merged.shape

(57, 14)

In [113]:
ma_merged = ma_merged.astype({'Cluster Labels': int}, inplace = True, error = None)

In [114]:
ma_merged

,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Agawam,42.069539,-72.614812,1,Vietnamese Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
6,Amesbury,42.857954,-70.930092,0,American Restaurant,Chinese Restaurant,Comfort Food Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cambodian Restaurant,Caribbean Restaurant,Belgian Restaurant
16,Attleboro,41.944544,-71.285608,1,Asian Restaurant,Spanish Restaurant,Italian Restaurant,Vietnamese Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant
20,Barnstable,41.700113,-70.299467,0,American Restaurant,Seafood Restaurant,Vietnamese Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant
30,Beverly,42.558428,-70.880049,1,Vietnamese Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
35,Boston,42.360253,-71.058291,1,Seafood Restaurant,Italian Restaurant,American Restaurant,Restaurant,Falafel Restaurant,Greek Restaurant,Mediterranean Restaurant,New American Restaurant,Thai Restaurant,Belgian Restaurant
40,Braintree,42.222322,-70.999492,1,Vietnamese Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
42,Bridgewater,41.990379,-70.975043,0,American Restaurant,Chinese Restaurant,Thai Restaurant,Italian Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant
44,Brockton,42.083433,-71.018379,1,Asian Restaurant,Vietnamese Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant
49,Cambridge,42.375100,-71.105616,1,Indian Restaurant,New American Restaurant,Middle Eastern Restaurant,Thai Restaurant,Tapas Restaurant,Italian Restaurant,Mediterranean Restaurant,Mexican Restaurant,French Restaurant,Fast Food Restaurant


### Visualize the Clusters

In [122]:
# Create map

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ma_merged['Latitude'], 
                                  ma_merged['Longitude'], 
                                  ma_merged['City'], 
                                  ma_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Check the Clusters Seperately

In [116]:
ma_label0 = ma_merged.loc[ma_merged['Cluster Labels'] == 0, ma_merged.columns[[0] + list(range(1, ma_merged.shape[1]))]]
ma_label0

,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Amesbury,42.857954,-70.930092,0,American Restaurant,Chinese Restaurant,Comfort Food Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cambodian Restaurant,Caribbean Restaurant,Belgian Restaurant
20,Barnstable,41.700113,-70.299467,0,American Restaurant,Seafood Restaurant,Vietnamese Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant
42,Bridgewater,41.990379,-70.975043,0,American Restaurant,Chinese Restaurant,Thai Restaurant,Italian Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant
97,Fitchburg,42.583423,-71.802296,0,American Restaurant,Chinese Restaurant,Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant
169,Marlborough,42.345927,-71.552287,0,American Restaurant,Chinese Restaurant,Thai Restaurant,Seafood Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant
205,Newburyport,42.812591,-70.877275,0,American Restaurant,Seafood Restaurant,Italian Restaurant,New American Restaurant,Vietnamese Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant
313,Watertown,42.370930,-71.182832,0,Persian Restaurant,American Restaurant,Middle Eastern Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant


In [133]:
#Create map for cluster label = 0
map_clusters0 = folium.Map(location=[latitude, longitude], zoom_start=7)
# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ma_label0['Latitude'], 
                                  ma_label0['Longitude'], 
                                  ma_label0['City'], 
                                  ma_label0['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters0)
       
map_clusters0

In [124]:
ma_label1 = ma_merged.loc[ma_merged['Cluster Labels'] == 1, ma_merged.columns[[0] + list(range(1, ma_merged.shape[1]))]]
ma_label1

,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Agawam,42.069539,-72.614812,1,Vietnamese Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
16,Attleboro,41.944544,-71.285608,1,Asian Restaurant,Spanish Restaurant,Italian Restaurant,Vietnamese Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant
30,Beverly,42.558428,-70.880049,1,Vietnamese Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
35,Boston,42.360253,-71.058291,1,Seafood Restaurant,Italian Restaurant,American Restaurant,Restaurant,Falafel Restaurant,Greek Restaurant,Mediterranean Restaurant,New American Restaurant,Thai Restaurant,Belgian Restaurant
40,Braintree,42.222322,-70.999492,1,Vietnamese Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
44,Brockton,42.083433,-71.018379,1,Asian Restaurant,Vietnamese Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant
49,Cambridge,42.375100,-71.105616,1,Indian Restaurant,New American Restaurant,Middle Eastern Restaurant,Thai Restaurant,Tapas Restaurant,Italian Restaurant,Mediterranean Restaurant,Mexican Restaurant,French Restaurant,Fast Food Restaurant
61,Chicopee,42.148704,-72.607867,1,German Restaurant,Vietnamese Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
85,East Longmeadow,42.064540,-72.512586,1,American Restaurant,Mexican Restaurant,Vietnamese Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant
87,Easthampton,42.266757,-72.668980,1,Indian Restaurant,Mexican Restaurant,Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant


In [125]:
#Create map for cluster label = 1
map_clusters1 = folium.Map(location=[latitude, longitude], zoom_start=7)
# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ma_label1['Latitude'], 
                                  ma_label1['Longitude'], 
                                  ma_label1['City'], 
                                  ma_label1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='purple',
        fill_opacity=0.7).add_to(map_clusters1)
       
map_clusters1

In [126]:
ma_label2 = ma_merged.loc[ma_merged['Cluster Labels'] == 2, ma_merged.columns[[0] + list(range(1, ma_merged.shape[1]))]]
ma_label2

,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
164,Malden,42.425096,-71.066163,2,Asian Restaurant,Chinese Restaurant,Vietnamese Restaurant,American Restaurant,Indian Restaurant,Korean Restaurant,Brazilian Restaurant,Ethiopian Restaurant,Italian Restaurant,Japanese Restaurant
175,Medford,42.418430,-71.106164,2,Italian Restaurant,American Restaurant,Chinese Restaurant,Thai Restaurant,Restaurant,Mexican Restaurant,Asian Restaurant,Japanese Restaurant,Cambodian Restaurant,Cajun / Creole Restaurant
177,Melrose,42.456432,-71.064182,2,Italian Restaurant,American Restaurant,Sushi Restaurant,Seafood Restaurant,Mexican Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant
213,Northampton,42.319023,-72.630634,2,Italian Restaurant,Vegetarian / Vegan Restaurant,Indian Restaurant,American Restaurant,Thai Restaurant,Asian Restaurant,Sushi Restaurant,Greek Restaurant,Halal Restaurant,Moroccan Restaurant
228,Peabody,42.527873,-70.928661,2,Italian Restaurant,Sushi Restaurant,Mexican Restaurant,Restaurant,Portuguese Restaurant,Comfort Food Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant
235,Pittsfield,42.450097,-73.245379,2,Italian Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
242,Quincy,42.252877,-71.002270,2,American Restaurant,Italian Restaurant,Mediterranean Restaurant,Vietnamese Restaurant,Chinese Restaurant,Korean Restaurant,Mexican Restaurant,New American Restaurant,Cajun / Creole Restaurant,Brazilian Restaurant
243,Randolph,42.161800,-71.042484,2,Vietnamese Restaurant,Italian Restaurant,American Restaurant,Ethiopian Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant
247,Revere,42.408430,-71.011995,2,Italian Restaurant,American Restaurant,Mexican Restaurant,Vietnamese Restaurant,Ethiopian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant
307,Waltham,42.375640,-71.235800,2,Italian Restaurant,Sushi Restaurant,Mexican Restaurant,African Restaurant,Chinese Restaurant,Latin American Restaurant,Mediterranean Restaurant,Cuban Restaurant,Greek Restaurant,Thai Restaurant


In [127]:
#Create map for cluster label = 2
map_clusters2 = folium.Map(location=[latitude, longitude], zoom_start=7)
# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ma_label2['Latitude'], 
                                  ma_label2['Longitude'], 
                                  ma_label2['City'], 
                                  ma_label2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_clusters2)
       
map_clusters2

In [131]:
ma_label3 = ma_merged.loc[ma_merged['Cluster Labels'] == 3, ma_merged.columns[[0] + list(range(1, ma_merged.shape[1]))]]
ma_label3

,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
162,Lynn,42.466763,-70.949494,3,Mexican Restaurant,American Restaurant,Asian Restaurant,Caribbean Restaurant,Chinese Restaurant,Italian Restaurant,New American Restaurant,Vietnamese Restaurant,Falafel Restaurant,Halal Restaurant
180,Methuen,42.726202,-71.190892,3,Mexican Restaurant,Restaurant,Chinese Restaurant,Vietnamese Restaurant,Cuban Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Fast Food Restaurant


In [134]:
#Create map for cluster label = 3
map_clusters3 = folium.Map(location=[latitude, longitude], zoom_start=9)
# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ma_label3['Latitude'], 
                                  ma_label3['Longitude'], 
                                  ma_label3['City'], 
                                  ma_label3['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7).add_to(map_clusters3)
       
map_clusters3

In [136]:
ma_label4 = ma_merged.loc[ma_merged['Cluster Labels'] == 4, ma_merged.columns[[0] + list(range(1, ma_merged.shape[1]))]]
ma_label4

,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,Chelsea,42.391764,-71.032828,4,Thai Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Comfort Food Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Falafel Restaurant,Ethiopian Restaurant
200,New Bedford,41.636215,-70.934205,4,Fast Food Restaurant,Vietnamese Restaurant,Cuban Restaurant,Indian Restaurant,Halal Restaurant,Greek Restaurant,German Restaurant,French Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [137]:
#Create map for cluster label = 4
map_clusters4 = folium.Map(location=[latitude, longitude], zoom_start=8)
# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ma_label4['Latitude'], 
                                  ma_label4['Longitude'], 
                                  ma_label4['City'], 
                                  ma_label4['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7).add_to(map_clusters4)
       
map_clusters4